In [1]:
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import datetime as dt
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GroupKFold
import seaborn as sns
import statistics
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LassoLars
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
import datetime as dt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [2]:
#importing MAC season 4 - days to floering trait file
flowering=pd.read_csv("/Users/ishitadebnath/Desktop/Genomics/MAC season 4/Phenotypes/Modified/mac_season_4_days_gdd_to_flowering.csv" , delimiter=",")
flowering=flowering.rename(columns={"genotype": "cultivar"})

In [3]:
#Importing and merging the gene and trait dataset
dist=pd.read_csv("/Users/ishitadebnath/Desktop/Genomics/cluster.txt", delimiter="\t")
a=dist.X
dist=dist.rename(columns={"X": "cultivar"})
dist=dist.dropna()

flowering=pd.merge(flowering, dist, on="cultivar", how='inner')
print(flowering)

                                              plot  range  column  \
0     MAC Field Scanner Season 4 Range 51 Column 4     51       4   
1     MAC Field Scanner Season 4 Range 10 Column 8     10       8   
2     MAC Field Scanner Season 4 Range 16 Column 7     16       7   
3    MAC Field Scanner Season 4 Range 35 Column 11     35      11   
4     MAC Field Scanner Season 4 Range 50 Column 7     50       7   
..                                             ...    ...     ...   
116   MAC Field Scanner Season 4 Range 46 Column 9     46       9   
117   MAC Field Scanner Season 4 Range 11 Column 8     11       8   
118  MAC Field Scanner Season 4 Range 50 Column 14     50      14   
119  MAC Field Scanner Season 4 Range 20 Column 10     20      10   
120  MAC Field Scanner Season 4 Range 50 Column 12     50      12   

      scientificname  cultivar  \
0    Sorghum bicolor  PI586435   
1    Sorghum bicolor  PI586435   
2    Sorghum bicolor  PI152651   
3    Sorghum bicolor  PI152651   
4

In [4]:
#Creating a feature space and standardizing the features. 

d=flowering.drop(columns=['cultivar','days_to_flowering',  'treatment', 'blocking_height',
       'scientificname','method_type', 'date_of_flowering', 'gdd_to_flowering', 'method','plot','range','column'])

X=d.to_numpy()
min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

y = flowering[['days_to_flowering']]
y = np.asarray(y).flatten()

In [5]:
# Hyper parameter tuning

In [6]:
model = xgb.XGBRegressor()
groups=flowering.cultivar
n_estimators = range(10, 60, 5)
learning_rate = np.linspace(0,1,11)
max_depth = range(1, 20, 2)
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators, learning_rate=learning_rate)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

[22:43:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best: -10.526790 using {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 20}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 10}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 15}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 20}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 25}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 30}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 35}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 40}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 45}
-68.834672 (3.334351) with: {'learning_rate': 0.0, 'max_depth': 1, 'n_estimators': 50}
-68.834672 (3.33435

In [7]:
model = RandomForestRegressor()
max_depth = range(1, 20, 2)
n_estimators=range(20, 55, 5)
groups=flowering.cultivar
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -10.551498 using {'max_depth': 1, 'n_estimators': 35}
-10.592442 (1.603365) with: {'max_depth': 1, 'n_estimators': 20}
-10.839299 (1.632040) with: {'max_depth': 1, 'n_estimators': 25}
-10.814157 (1.476718) with: {'max_depth': 1, 'n_estimators': 30}
-10.551498 (1.479566) with: {'max_depth': 1, 'n_estimators': 35}
-10.791624 (1.478289) with: {'max_depth': 1, 'n_estimators': 40}
-10.961436 (1.478733) with: {'max_depth': 1, 'n_estimators': 45}
-10.707392 (1.537382) with: {'max_depth': 1, 'n_estimators': 50}
-11.184354 (1.533559) with: {'max_depth': 3, 'n_estimators': 20}
-11.076723 (1.492975) with: {'max_depth': 3, 'n_estimators': 25}
-11.021299 (1.599161) with: {'max_depth': 3, 'n_estimators': 30}
-10.791087 (1.786236) with: {'max_depth': 3, 'n_estimators': 35}
-10.836509 (1.457004) with: {'max_depth': 3, 'n_estimators': 40}
-10.756605 (1.694239) with: {'max_depth': 3, 'n_estimators': 45}
-10.935023 (1.463645) with: {'max_depth': 3, 'n_estimators': 50}
-10.944052 (1.998693) with: {'

In [8]:
model = SGDRegressor()
alpha = np.arange(0.01, 1.0, 0.05)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))



Best: -11.038267 using {'alpha': 0.91}
-11.363838 (1.915043) with: {'alpha': 0.01}
-11.481518 (1.978059) with: {'alpha': 0.060000000000000005}
-11.460419 (1.612300) with: {'alpha': 0.11}
-11.325217 (1.490394) with: {'alpha': 0.16000000000000003}
-11.611242 (2.010160) with: {'alpha': 0.21000000000000002}
-11.361391 (1.601717) with: {'alpha': 0.26}
-11.688397 (1.725833) with: {'alpha': 0.31000000000000005}
-11.710945 (1.907395) with: {'alpha': 0.36000000000000004}
-11.330368 (1.727497) with: {'alpha': 0.41000000000000003}
-11.317342 (1.682243) with: {'alpha': 0.46}
-11.562832 (2.034565) with: {'alpha': 0.51}
-11.251936 (1.824433) with: {'alpha': 0.56}
-11.145701 (1.679855) with: {'alpha': 0.6100000000000001}
-11.209092 (1.813007) with: {'alpha': 0.66}
-11.232983 (1.254530) with: {'alpha': 0.7100000000000001}
-11.047175 (1.593518) with: {'alpha': 0.76}
-11.093779 (1.588492) with: {'alpha': 0.81}
-11.550986 (1.813170) with: {'alpha': 0.8600000000000001}
-11.038267 (1.529801) with: {'alpha'

In [9]:
model = LassoLars()
alpha = np.arange(0.1, 1.0, 0.01)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -10.644825 using {'alpha': 0.1}
-10.644825 (1.366980) with: {'alpha': 0.1}
-10.667549 (1.321140) with: {'alpha': 0.11}
-10.701021 (1.272772) with: {'alpha': 0.12}
-10.739300 (1.225651) with: {'alpha': 0.13}
-10.759864 (1.195519) with: {'alpha': 0.13999999999999999}
-10.768697 (1.162553) with: {'alpha': 0.14999999999999997}
-10.776695 (1.129323) with: {'alpha': 0.15999999999999998}
-10.778428 (1.106686) with: {'alpha': 0.16999999999999998}
-10.779920 (1.093990) with: {'alpha': 0.17999999999999997}
-10.784493 (1.086640) with: {'alpha': 0.18999999999999995}
-10.791930 (1.084175) with: {'alpha': 0.19999999999999996}
-10.800641 (1.083945) with: {'alpha': 0.20999999999999996}
-10.808781 (1.084961) with: {'alpha': 0.21999999999999995}
-10.820485 (1.092062) with: {'alpha': 0.22999999999999995}
-10.834268 (1.101044) with: {'alpha': 0.23999999999999994}
-10.848983 (1.112114) with: {'alpha': 0.24999999999999992}
-10.864508 (1.125214) with: {'alpha': 0.2599999999999999}
-10.880843 (1.140243)

In [10]:
model = Lasso()
alpha = np.arange(0.1, 1.0, 0.01)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -10.521031 using {'alpha': 0.12}
-10.528280 (1.519138) with: {'alpha': 0.1}
-10.524125 (1.510668) with: {'alpha': 0.11}
-10.521031 (1.510869) with: {'alpha': 0.12}
-10.526212 (1.511970) with: {'alpha': 0.13}
-10.538711 (1.507123) with: {'alpha': 0.13999999999999999}
-10.542722 (1.490006) with: {'alpha': 0.14999999999999997}
-10.546239 (1.475122) with: {'alpha': 0.15999999999999998}
-10.546228 (1.464803) with: {'alpha': 0.16999999999999998}
-10.549366 (1.453897) with: {'alpha': 0.17999999999999997}
-10.554785 (1.442934) with: {'alpha': 0.18999999999999995}
-10.561942 (1.432138) with: {'alpha': 0.19999999999999996}
-10.567453 (1.424438) with: {'alpha': 0.20999999999999996}
-10.574492 (1.416946) with: {'alpha': 0.21999999999999995}
-10.581200 (1.411343) with: {'alpha': 0.22999999999999995}
-10.585809 (1.403133) with: {'alpha': 0.23999999999999994}
-10.586549 (1.391582) with: {'alpha': 0.24999999999999992}
-10.588976 (1.379606) with: {'alpha': 0.2599999999999999}
-10.592964 (1.367362

In [11]:
model = Ridge()
alpha = np.arange(0.1, 1.0, 0.01)

groups=flowering.cultivar
param_grid = dict(alpha=alpha)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -10.455779 using {'alpha': 0.9899999999999995}
-11.393090 (2.503222) with: {'alpha': 0.1}
-11.313869 (2.475830) with: {'alpha': 0.11}
-11.245112 (2.449534) with: {'alpha': 0.12}
-11.184923 (2.424317) with: {'alpha': 0.13}
-11.131835 (2.400149) with: {'alpha': 0.13999999999999999}
-11.084695 (2.376995) with: {'alpha': 0.14999999999999997}
-11.042586 (2.354816) with: {'alpha': 0.15999999999999998}
-11.004768 (2.333570) with: {'alpha': 0.16999999999999998}
-10.970637 (2.313212) with: {'alpha': 0.17999999999999997}
-10.939697 (2.293699) with: {'alpha': 0.18999999999999995}
-10.911534 (2.274990) with: {'alpha': 0.19999999999999996}
-10.885801 (2.257042) with: {'alpha': 0.20999999999999996}
-10.862208 (2.239816) with: {'alpha': 0.21999999999999995}
-10.840506 (2.223274) with: {'alpha': 0.22999999999999995}
-10.820483 (2.207380) with: {'alpha': 0.23999999999999994}
-10.801957 (2.192101) with: {'alpha': 0.24999999999999992}
-10.784770 (2.177403) with: {'alpha': 0.2599999999999999}
-10.76

In [12]:
model = ElasticNet()
alpha = np.arange(0.01, 1.0, 0.05)
l1_ratio=np.arange(0, 1, 0.01)
groups=flowering.cultivar
param_grid = dict(alpha=alpha, l1_ratio=l1_ratio)
kfold = GroupKFold(n_splits=5)
grid_search = GridSearchCV(model, param_grid, scoring="neg_root_mean_squared_error", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, y, groups)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -10.382837 using {'alpha': 0.060000000000000005, 'l1_ratio': 0.26}
-10.457984 (1.749444) with: {'alpha': 0.01, 'l1_ratio': 0.0}
-10.459197 (1.751339) with: {'alpha': 0.01, 'l1_ratio': 0.01}
-10.460438 (1.753280) with: {'alpha': 0.01, 'l1_ratio': 0.02}
-10.461695 (1.755245) with: {'alpha': 0.01, 'l1_ratio': 0.03}
-10.462986 (1.757233) with: {'alpha': 0.01, 'l1_ratio': 0.04}
-10.464280 (1.759226) with: {'alpha': 0.01, 'l1_ratio': 0.05}
-10.465622 (1.761281) with: {'alpha': 0.01, 'l1_ratio': 0.06}
-10.466984 (1.763355) with: {'alpha': 0.01, 'l1_ratio': 0.07}
-10.468353 (1.765487) with: {'alpha': 0.01, 'l1_ratio': 0.08}
-10.469813 (1.767732) with: {'alpha': 0.01, 'l1_ratio': 0.09}
-10.471256 (1.769933) with: {'alpha': 0.01, 'l1_ratio': 0.1}
-10.472728 (1.772171) with: {'alpha': 0.01, 'l1_ratio': 0.11}
-10.474065 (1.774188) with: {'alpha': 0.01, 'l1_ratio': 0.12}
-10.475436 (1.776260) with: {'alpha': 0.01, 'l1_ratio': 0.13}
-10.476759 (1.778409) with: {'alpha': 0.01, 'l1_ratio': 0.14}

In [13]:
#Implementation of different algorithms

# 5 fold cross-validation for Random Forest
groups=flowering.cultivar
ns=5
cv = GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X, y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print('TRAIN :',X_train.shape, 'TEST :', X_test.shape)
    crf=RandomForestRegressor(max_depth=19, n_estimators=35).fit(X_train, y_train)

    pred=crf.predict(X_test)
    error=error+math.sqrt(np.sum((pred-y_test)**2)/y_test.shape[0])
    R2=R2+r2_score(y_test, pred)
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    error=error+math.sqrt(mean_squared_error(y_test, pred))

std= (statistics.stdev(e))
print('R^2 for 5-fold CV RF:', R2/ns)
print('Root Mean square error for 5-fold CV RF',error/ns,'+-', std)

# 5 fold cross-validation for Stochastic Gradient Descent Regression

cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=SGDRegressor(alpha=0.91).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV SG:', R2/ns)
print('Root Mean square error for 5-fold CV SG:',error/ns, '+-', std)

# 5 fold cross-validation for Linear Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=LinearRegression().fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV LR:', R2/ns)
print('Root Mean square error for 5-fold CV LR:',error/ns, '+-', std)

# 5 fold cross-validation for XgBoost Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=xgb.XGBRegressor(booster='gbtree',importance_type='gain',learning_rate= 0.2, max_depth= 3, n_estimators= 20).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV XG:', R2/ns)
print('Root Mean square error for 5-fold CV XG:',error/ns, '+-', std)

# 5 fold cross-validation for LassoLars Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=LassoLars(alpha=0.1).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV LL:', R2/ns)
print('Root Mean square error for 5-fold CV LL:',error/ns, '+-', std)

# 5 fold cross-validation for Lasso Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=Lasso(alpha=0.12).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV L:', R2/ns)
print('Root Mean square error for 5-fold CV L:',error/ns, '+-', std)

# 5 fold cross-validation for Ridge Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=Ridge(alpha=0.99).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV R:', R2/ns)
print('Root Mean square error for 5-fold CV R:',error/ns, '+-', std)

# 5 fold cross-validation for ElasticNet Regression
cv =GroupKFold(n_splits=ns)
error=0
R2=0
e=[]
for train_index, test_index in cv.split(X,y,groups):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clm=ElasticNet(alpha= 0.06, l1_ratio= 0.26, max_iter=2000).fit(X_train, y_train)
    pred=clm.predict(X_test)
    error=error+math.sqrt(mean_squared_error(y_test, pred))
    e.append(math.sqrt(mean_squared_error(y_test, pred)))
    R2=R2+r2_score(y_test, pred)
    
std= (statistics.stdev(e))

print('R^2 for 5-fold CV EN:', R2/ns)
print('Root Mean square error for 5-fold CV EN:',error/ns, '+-', std)



TRAIN : (96, 30) TEST : (25, 30)
TRAIN : (97, 30) TEST : (24, 30)
TRAIN : (97, 30) TEST : (24, 30)
TRAIN : (97, 30) TEST : (24, 30)
TRAIN : (97, 30) TEST : (24, 30)
R^2 for 5-fold CV RF: -0.11841132076368614
Root Mean square error for 5-fold CV RF 21.886663850349688 +- 1.8466796823536122
R^2 for 5-fold CV SG: -0.12527536938380793
Root Mean square error for 5-fold CV SG: 11.015980986634913 +- 1.7931657662125908
R^2 for 5-fold CV LR: -1.1738248411545062
Root Mean square error for 5-fold CV LR: 14.894458908533718 +- 3.5133165741489334
[22:43:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:43:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror